In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score
import statsmodels.formula.api as smf

In [ ]:
df = pd.read_csv("RRCA_baseflow.csv")
# display(df)
df['Date'] = df['Date'] - 693963
df['Date_YMD'] = pd.to_datetime(df['Date'], origin='1900-01-01', unit='D')
df['Month'] = df['Date_YMD'].dt.month
# df = df[df['Observed']  0]
df = df.dropna()
def labelSeason(date):
    month = date.month
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

# Apply the function to create the 'season' column
df['season'] = df['Date_YMD'].apply(labelSeason)

df = df[~df["Segment_id"].isin([239, 256])]

# Display the dataframe to check the new column
display(df.head())
display(df)
# df.to_csv('RRCA_baseflow_YMD.csv')

# df_train = df[(df['Date_YMD'] >= '1941-01-01') & (df['Date_YMD'] <= '1989-12-31')]
# df_test = df[df['Date_YMD'] >= '1990-01-01']

# Display the first few rows of each dataframe
# print("Data from 1941 to 1989")
# display(df_train.head())

# print("\nData from 1990 to most recent")
# display(df_test.head())

In [ ]:
print(df.isnull().sum())

In [ ]:
unique_segment_ids = df['Segment_id'].unique()
print(unique_segment_ids)

In [ ]:
df.shape

In [ ]:
predictors = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping']
target = 'Observed'


for season in df['season'].unique():
    print(f"\nSeason: {season}")

    df_season = df[df['season'] == season]

    X_season = df_season[predictors]
    y_season = df_season[target]

    X_train, X_test, y_train, y_test = train_test_split(X_season, y_season, test_size=0.2, random_state=42)

    lm = LinearRegression()
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)

    r2_season = r2_score(y_test, y_pred)
    print("________Sklearn_______")
    print(f"Season {season} R-squared: {r2_season:.4f}")
    print("Intercept:", lm.intercept_)
    print("Coefficients:", lm.coef_)

    formula = 'Observed ~ Evapotranspiration + Precipitation + Irrigation_pumping'
    lms = smf.ols(formula=formula, data=df_season).fit()
    print("________Sklearn_______")
    print("\nCoefficients:")
    display(lms.params)

    print("\nConfidence Intervals:")
    display(lms.conf_int())

    print("\nModel Summary:")
    print(lms.summary())

    print("\nP-values:")
    display(lms.pvalues)

    plt.figure(figsize=(16, 12))
    for i, col in enumerate(predictors, 1):
        plt.subplot(2, 3, i)
        sns.regplot(data=df_season, x=col, y='Observed', scatter=True, line_kws={"color": "red"})
        plt.title(f'{col} vs Observed Baseflow ({season})')
    plt.tight_layout()
    plt.show()

    predictors_with_observed = predictors + [target]

    plt.figure(figsize=(16, 10))
    for i, col in enumerate(predictors_with_observed, 1):
        plt.subplot(2, 3, i)
        sns.histplot(df_season[col], kde=True)
        plt.title(f'Distribution of {col} ({season})')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(30, 8))
    for i, col in enumerate(predictors_with_observed, 1):
        plt.subplot(1, 4, i)
        sns.lineplot(data=df_season, x='Month', y=col, estimator='mean', errorbar=None, color='blue')
        plt.title(f'Monthly Average of {col} ({season})')
        plt.xlabel('Month')
        plt.ylabel(col)
        plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
plt.figure(figsize=(16, 12))

predictors = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping']
for i, col in enumerate(predictors, 1):
    plt.subplot(2, 3, i)
    sns.scatterplot(data=df, x=col, y='Observed')
    plt.title(f'{col} vs Observed Baseflow')
plt.tight_layout()
plt.show()

predictors_with_observed = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping', 'Observed']
plt.figure(figsize=(16, 10))
for i, col in enumerate(predictors_with_observed, 1):
    plt.subplot(2, 3, i)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

plt.figure(figsize=(30, 8))
for i, col in enumerate(predictors_with_observed, 1):
    plt.subplot(1, 4, i)
    sns.lineplot(data=df, x='Month', y=col, estimator='mean', errorbar=None, color='blue')
    plt.title(f'Monthly Average of {col}')
    plt.xlabel('Month')
    plt.ylabel(col)
    plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
means = df.groupby(['Segment_id']).agg({'Observed': 'mean'}).reset_index()
means.columns = ['SegmentID', 'Observed']
means_plot = sns.barplot(means, x='SegmentID', y='Observed')
means_plot.tick_params(axis='x', rotation=90) # Change the label direction
plt.show()

In [ ]:
# visualize the relationship between the features and the response using scatterplots
fig, axs = plt.subplots(1, 3, sharey=True)
df.plot(kind='scatter', x='Evapotranspiration', y='Observed', ax=axs[0], figsize=(16, 8))
df.plot(kind='scatter', x='Precipitation', y='Observed', ax=axs[1])
df.plot(kind='scatter', x='Irrigation_pumping', y='Observed', ax=axs[2])

In [ ]:
df.plot('Date_YMD', 'Observed')

In [ ]:
df.plot('Date_YMD', 'Irrigation_pumping')

In [ ]:
sns.scatterplot(df, x='Date_YMD', y='Observed', hue="Segment_id")

In [ ]:
display(df[df['Segment_id'] == 239])
display(df[df['Segment_id'] == 256])

In [13]:
# # create X and y
# feature_cols = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping', 'season']

# X = df[feature_cols]
# y = df.Observed

# # instantiate, fit
# lm = LinearRegression()
# lm.fit(X, y)

# # print coefficients
# x = zip(feature_cols, lm.coef_)
# for y in x:
#     display(y)

In [14]:
# lm = smf.ols(formula='Observed ~ Irrigation_pumping', data=df[df['season'] == 0]).fit()
# # print the coefficients
# display(lm.params)

# # print the confidence intervals for the model coefficients
# display(lm.conf_int())

In [15]:
# specific_segment_id = 144 
# df_specific_segment = df[df['Segment_id'] == specific_segment_id]
# print(f"Data for Segment id {specific_segment_id}:")
# display(df_specific_segment.head())
# # df_specific_segment.to_csv(f'{specific_segment_id}segment.csv')

# plt.figure(figsize=(16, 12))
# predictors = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping']

# for i, col in enumerate(predictors, 1):
#     plt.subplot(2, 3, i)
#     sns.scatterplot(data=df_specific_segment, x=col, y='Observed')
#     plt.title(f'{col} vs Observed Baseflow')
# plt.tight_layout()
# plt.show()

# plt.figure(figsize=(30, 8))
# predictors_with_observed = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping','Observed']
# for i, col in enumerate(predictors_with_observed, 1):
#     plt.subplot(1, 4, i)
#     sns.scatterplot(data=df_specific_segment, x='Date_YMD', y=col, label=col, color='blue', alpha=0.6)
#     plt.title(f'{col} Over Time')
#     plt.xlabel('Date')
#     plt.ylabel(col)
#     plt.xticks(rotation=45)
#     plt.grid(True)
# plt.tight_layout()
# plt.show()

# plt.figure(figsize=(16, 10))
# for i, col in enumerate(predictors_with_observed, 1):
#     plt.subplot(2, 3, i)
#     sns.histplot(df_specific_segment[col], kde=True)
#     plt.title(f'Distribution of {col}')

# plt.tight_layout()
# plt.show()

In [16]:
# specific_segment_id = 256 
# df_specific_segment = df[df['Segment_id'] == specific_segment_id]
# print(f"Data for Segment id {specific_segment_id}:")
# display(df_specific_segment.head())
# # df_specific_segment.to_csv(f'{specific_segment_id}segment.csv')

# predictors = ['Evapotranspiration']
# target = 'Observed'

# plt.figure(figsize=(16, 12))
# for i, col in enumerate(predictors, 1):
#     plt.subplot(2, 3, i)
#     sns.scatterplot(data=df_specific_segment, x=col, y='Observed')
#     plt.title(f'{col} vs Observed Baseflow')
# plt.tight_layout()
# plt.show()

# plt.figure(figsize=(30, 8))
# predictors_with_observed = ['Evapotranspiration', 'Observed']

# for i, col in enumerate(predictors_with_observed, 1):
#     plt.subplot(1, 4, i)
#     sns.scatterplot(data=df_specific_segment, x='Date_YMD', y=col, label=col, color='blue', alpha=0.6)
#     plt.title(f'{col} Over Time')
#     plt.xlabel('Date')
#     plt.ylabel(col)
#     plt.xticks(rotation=45)
#     plt.grid(True)
# plt.tight_layout()
# plt.show()

# plt.figure(figsize=(16, 10))
# for i, col in enumerate(predictors_with_observed, 1):
#     plt.subplot(2, 3, i)
#     sns.histplot(df_specific_segment[col], kde=True)
#     plt.title(f'Distribution of {col}')
# plt.tight_layout()
# plt.show()

# plt.figure(figsize=(30, 8))
# for i, col in enumerate(predictors_with_observed, 1):
#     plt.subplot(1, 4, i)
#     sns.lineplot(data=df_specific_segment, x='Month', y=col, estimator='mean', errorbar=None, color='blue')
#     plt.title(f'Monthly Average of {col}')
#     plt.xlabel('Month')
#     plt.ylabel(col)
#     plt.grid(True)
# plt.tight_layout()
# plt.show()

# x = df_specific_segment[predictors].values
# y = df_specific_segment[target].values

# lm = LinearRegression()
# lm.fit(x, y)
# print(lm.intercept_)
# print(lm.coef_)

# formula = 'Observed ~ Evapotranspiration + Precipitation + Irrigation_pumping'
# lms = smf.ols(formula=formula, data=df_specific_segment).fit()

# print("Coefficients")
# display(lms.params)

# print("Confidence Intervals")
# display(lms.conf_int())

# print("Model Summary")
# print(lms.summary())

# print("P-values")
# display(lms.pvalues)


#### This looks at data and distribution over time per segment

In [17]:
# for segment in unique_segment_ids:
#     specific_segment_id = segment
#     df_specific_segment = df[df['Segment_id'] == specific_segment_id]
#     print(f"Data for Segment id {specific_segment_id}:")
#     # display(df_specific_segment.head())
#     # df_specific_segment.to_csv(f'{specific_segment_id}segment.csv')
#     predictors = ['Evapotranspiration']
#     target = ['Observed']

#     # plt.figure(figsize=(16, 12))
#     # for i, col in enumerate(predictors, 1):
#     #     plt.subplot(2, 3, i)
#     #     sns.scatterplot(data=df_specific_segment, x=col, y='Observed')
#     #     plt.title(f'{col} vs Observed Baseflow')
#     # plt.tight_layout()
#     # plt.show()

#     plt.figure(figsize=(30, 8))
#     predictors_with_observed = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping','Observed']

#     # for i, col in enumerate(predictors_with_observed, 1):
#     #     plt.subplot(1, 4, i)
#     #     sns.scatterplot(data=df_specific_segment, x='Date_YMD', y=col, label=col, color='blue', alpha=0.6)
#     #     plt.title(f'{col} Over Time')
#     #     plt.xlabel('Date')
#     #     plt.ylabel(col)
#     #     plt.xticks(rotation=45)
#     #     plt.grid(True)
#     # plt.tight_layout()
#     # plt.show()

#     # plt.figure(figsize=(16, 10))
#     # for i, col in enumerate(predictors_with_observed, 1):
#     #     plt.subplot(2, 3, i)
#     #     sns.histplot(df_specific_segment[col], kde=True)
#     #     plt.title(f'Distribution of {col}')
#     # plt.tight_layout()
#     # plt.show()

#     # plt.figure(figsize=(30, 8))
#     # for i, col in enumerate(predictors_with_observed, 1):
#     #     plt.subplot(1, 4, i)
#     #     sns.lineplot(data=df_specific_segment, x='Month', y=col, estimator='mean', color='blue')
#     #     plt.title(f'Monthly Average of {col}')
#     #     plt.xlabel('Month')
#     #     plt.ylabel(col)
#     #     plt.grid(True)
#     # plt.tight_layout()
#     # plt.show()
    
#     # plt.figure(figsize=(30, 8))
#     # for i, col in enumerate(predictors_with_observed, 1):
#     #     plt.subplot(1, 4, i)
#     #     sns.scatterplot(data=df_specific_segment, x='Month', y=col, color='blue', alpha=0.6)
#     #     plt.title(f'{col} by Month')
#     #     plt.xlabel('Month')
#     #     plt.ylabel(col)
#     #     plt.grid(True)
#     # plt.tight_layout()
#     # plt.show()

#     x = df_specific_segment[predictors].values
#     y = df_specific_segment[target].values.ravel()

#     x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#     print("____Sklearn____")
#     lm = LinearRegression()
#     lm.fit(x_train, y_train)
#     print("Intercept")
#     print(lm.intercept_)
#     print("Coefficents")
#     print(lm.coef_)
#     print("Predictions on test sample")
#     display(lm.predict(x_test))

#     print("____Statsmodel____")
#     formula = 'Observed ~ Evapotranspiration + Precipitation + Irrigation_pumping'
#     lms = smf.ols(formula=formula, data=df_specific_segment).fit()
#     print("Coefficients")
#     display(lms.params)
#     print("Confidence Intervals")
#     display(lms.conf_int())
#     print("Model Summary")
#     print(lms.summary())
#     print("P-values")
#     display(lms.pvalues)